In [1]:
import pandas as pd

merged_df = pd.read_csv('bixi_data_2023.csv')
for i in range(2022, 2023):
    df = pd.read_csv('bixi_data_' + str(i) + '.csv')
    # Specify the column for comparison
    column_to_compare = 'STARTSTATIONNAME'
    mask = df[column_to_compare].isin(merged_df[column_to_compare])

    # Apply the mask to df2 to keep only the rows present in df1
    df = df[mask]
    merged_df = pd.concat([merged_df, df], ignore_index=True)

merged_df.head(100)


FileNotFoundError: [Errno 2] No such file or directory: 'bixi_data_2023.csv'

In [ ]:
merged_df.shape

(19484411, 10)

In [ ]:
#Convert timestamps to datetime objects
df_copy = merged_df.copy()
df_copy['STARTTIMETS'] = pd.to_datetime(df_copy['STARTTIMEMS'], unit='ms').dt.floor('s') #floor to nearest second
df_copy['ENDTIMETS'] = pd.to_datetime(df_copy['ENDTIMEMS'], unit='ms').dt.floor('s')

#split date and time columns
df_copy['STARTDATE'] = df_copy['STARTTIMETS'].dt.date
df_copy['STARTTIME'] = df_copy['STARTTIMETS'].dt.time
df_copy['ENDDATE'] = df_copy['ENDTIMETS'].dt.date
df_copy['ENDTIME'] = df_copy['ENDTIMETS'].dt.time

#group by intervals of 30 minutes
df_copy['ROUNDEDSTARTTIME'] = df_copy['STARTTIMETS'].dt.floor('30min')
df_copy['ROUNDEDENDTIME'] = df_copy['ENDTIMETS'].dt.floor('30min')

#Filter our data that is not in timeframe desired
start_month_day = '04-16'
end_month_day = '11-15'

# Extract month and day components and filter rows
df_copy = df_copy[
    (df_copy['ROUNDEDSTARTTIME'].dt.strftime('%m-%d') >= start_month_day) &
    (df_copy['ROUNDEDSTARTTIME'].dt.strftime('%m-%d') <= end_month_day)
]

#drop original timestamp columns
df_copy = df_copy.drop(columns=['STARTTIMEMS', 'ENDTIMEMS'])


df_copy.tail(100)

,STARTSTATIONNAME,STARTSTATIONARRONDISSEMENT,STARTSTATIONLATITUDE,STARTSTATIONLONGITUDE,ENDSTATIONNAME,ENDSTATIONARRONDISSEMENT,ENDSTATIONLATITUDE,ENDSTATIONLONGITUDE,STARTTIMETS,ENDTIMETS,STARTDATE,STARTTIME,ENDDATE,ENDTIME,ROUNDEDSTARTTIME,ROUNDEDENDTIME
19484311,Letourneux / Ste-Catherine,Mercier - Hochelaga-Maisonneuve,45.550784,-73.534100,de la Commune / Berri,Ville-Marie,45.510860,-73.549830,2022-08-20 16:47:58,2022-08-20 17:10:45,2022-08-20,16:47:58,2022-08-20,17:10:45,2022-08-20 16:30:00,2022-08-20 17:00:00
19484312,Casgrain / St-Viateur,Le Plateau-Mont-Royal,45.527513,-73.598791,de Gaspé / Beaubien,Rosemont - La Petite-Patrie,45.531958,-73.607001,2022-08-20 16:47:58,2022-08-20 16:52:50,2022-08-20,16:47:58,2022-08-20,16:52:50,2022-08-20 16:30:00,2022-08-20 16:30:00
19484313,Marché Jean-Talon (Henri-Julien / Jean-Talon),Rosemont - La Petite-Patrie,45.536785,-73.614888,St-Catherine / St-Laurent,Ville-Marie,45.509932,-73.563807,2022-08-20 16:48:03,2022-08-20 17:16:25,2022-08-20,16:48:03,2022-08-20,17:16:25,2022-08-20 16:30:00,2022-08-20 17:00:00
19484314,de la Commune / St-Sulpice,Ville-Marie,45.504242,-73.553470,Berri / Cherrier,Le Plateau-Mont-Royal,45.519088,-73.569509,2022-08-20 16:48:03,2022-08-20 17:14:07,2022-08-20,16:48:03,2022-08-20,17:14:07,2022-08-20 16:30:00,2022-08-20 17:00:00
19484315,Marquette / du Mont-Royal,Le Plateau-Mont-Royal,45.532290,-73.575440,Duluth / St-Laurent,Le Plateau-Mont-Royal,45.516876,-73.579460,2022-08-20 16:48:05,2022-08-20 17:01:34,2022-08-20,16:48:05,2022-08-20,17:01:34,2022-08-20 16:30:00,2022-08-20 17:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19484406,Parc du Pélican (1ère avenue / Masson),Rosemont - La Petite-Patrie,45.545200,-73.576451,Métro Beaubien (de Chateaubriand / Beaubien),Rosemont - La Petite-Patrie,45.535360,-73.603635,2022-08-20 16:50:08,2022-08-20 17:05:03,2022-08-20,16:50:08,2022-08-20,17:05:03,2022-08-20 16:30:00,2022-08-20 17:00:00
19484407,St-Charles / Thomas-Keefer,Le Sud-Ouest,45.477605,-73.573775,de la Commune / St-Sulpice,Ville-Marie,45.504242,-73.553470,2022-08-20 16:50:05,2022-08-20 17:20:35,2022-08-20,16:50:05,2022-08-20,17:20:35,2022-08-20 16:30:00,2022-08-20 17:00:00
19484408,de Lanaudière / Laurier,Le Plateau-Mont-Royal,45.533314,-73.583737,Duluth / St-Laurent,Le Plateau-Mont-Royal,45.516876,-73.579460,2022-08-20 16:50:16,2022-08-20 17:09:26,2022-08-20,16:50:16,2022-08-20,17:09:26,2022-08-20 16:30:00,2022-08-20 17:00:00
19484409,Prince-Arthur / du Parc,Le Plateau-Mont-Royal,45.510590,-73.575470,Vallières / St-Laurent,Le Plateau-Mont-Royal,45.518967,-73.583616,2022-08-20 16:50:13,2022-08-20 16:55:25,2022-08-20,16:50:13,2022-08-20,16:55:25,2022-08-20 16:30:00,2022-08-20 16:30:00


In [ ]:
df_copy_resampled = df_copy.resample('30T').ffill()


In [ ]:
#drop nan data
print("Before: " + str(df_copy.shape))
df_copy = df_copy.dropna()
print("After: " + str(df_copy.shape))


Before: (19034375, 16)
After: (18901703, 16)


In [ ]:
#split into incoming bikes
incoming_bikes = df_copy.copy()
incoming_bikes.drop(columns=incoming_bikes.columns.difference(['ENDSTATIONNAME', 'ROUNDEDENDTIME', 'ENDSTATIONARRONDISSEMENT']), inplace=True)
# grouped_df = incoming_bikes.groupby(['ROUNDEDENDTIME', 'ENDSTATIONNAME', 'ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE']).count()
incoming_bikes = incoming_bikes.groupby(['ENDSTATIONNAME', 'ROUNDEDENDTIME' ],as_index=False).count()
incoming_bikes = incoming_bikes.rename(columns={'ENDSTATIONARRONDISSEMENT': 'COUNTEVENTS'})

#insert extra columns
incoming_bikes['ENDDATE'] = incoming_bikes['ROUNDEDENDTIME'].dt.date
incoming_bikes['ENDDAYNUM'] = incoming_bikes['ROUNDEDENDTIME'].dt.day
incoming_bikes['ENDMONTHNUM'] = incoming_bikes['ROUNDEDENDTIME'].dt.month
incoming_bikes['ENDTIME'] = incoming_bikes['ROUNDEDENDTIME'].dt.time
incoming_bikes['ENDHOUR'] = incoming_bikes['ROUNDEDENDTIME'].dt.hour
incoming_bikes['ENDMINUTE'] = incoming_bikes['ROUNDEDENDTIME'].dt.minute
incoming_bikes['DAYOFWEEK'] = incoming_bikes['ROUNDEDENDTIME'].dt.dayofweek



# # incoming_bikes = incoming_bikes.groupby(['STARTSTATIONNAME', 'ENDDATE', interval]).count()

# #sort by ROUNDEDENDTIME
# outgoing_bikes = outgoing_bikes.sort_values(by=['ROUNDEDSTARTTIME'])
# outgoing_bikes = outgoing_bikes[outgoing_bikes['STARTSTATIONNAME'] == '5e avenue / Masson']

incoming_bikes.size



69375820

In [ ]:
#split into incoming bikes
outgoing_bikes = df_copy.copy()
outgoing_bikes.drop(columns=outgoing_bikes.columns.difference(['STARTSTATIONNAME', 'ROUNDEDSTARTTIME', 'STARTSTATIONARRONDISSEMENT']), inplace=True)
# grouped_df = incoming_bikes.groupby(['ROUNDEDENDTIME', 'ENDSTATIONNAME', 'ENDSTATIONLATITUDE', 'ENDSTATIONLONGITUDE']).count()
outgoing_bikes = outgoing_bikes.groupby(['STARTSTATIONNAME', 'ROUNDEDSTARTTIME' ],as_index=False).count()
outgoing_bikes = outgoing_bikes.rename(columns={'STARTSTATIONARRONDISSEMENT': 'COUNTEVENTS'})

#insert extra columns
outgoing_bikes['STARTDATE'] = outgoing_bikes['ROUNDEDSTARTTIME'].dt.date
outgoing_bikes['STARTDAYNUM'] = outgoing_bikes['ROUNDEDSTARTTIME'].dt.day
outgoing_bikes['STARTMONTHNUM'] = outgoing_bikes['ROUNDEDSTARTTIME'].dt.month
outgoing_bikes['STARTTIME'] = outgoing_bikes['ROUNDEDSTARTTIME'].dt.time
outgoing_bikes['STARTHOUR'] = outgoing_bikes['ROUNDEDSTARTTIME'].dt.hour
outgoing_bikes['STARTMINUTE'] = outgoing_bikes['ROUNDEDSTARTTIME'].dt.minute
outgoing_bikes['DAYOFWEEK'] = outgoing_bikes['ROUNDEDSTARTTIME'].dt.dayofweek



# # incoming_bikes = incoming_bikes.groupby(['STARTSTATIONNAME', 'ENDDATE', interval]).count()

# #sort by ROUNDEDENDTIME
# outgoing_bikes = outgoing_bikes.sort_values(by=['ROUNDEDSTARTTIME'])
# outgoing_bikes = outgoing_bikes[outgoing_bikes['STARTSTATIONNAME'] == '5e avenue / Masson']

outgoing_bikes.size


65783780

In [ ]:
print("Original data was " + str(df_copy.shape))
print("Incoming bikes data is " + str(incoming_bikes.shape))
print("Outgoing bikes data is " + str(outgoing_bikes.shape))
print("Combined incoming and outgoing bikes data is " + str(incoming_bikes.shape[0] + outgoing_bikes.shape[0]))



Original data was (18901703, 16)
Incoming bikes data is (6937582, 10)
Outgoing bikes data is (6578378, 10)
Combined incoming and outgoing bikes data is 13515960


In [ ]:
incoming_bikes.sample(100)

,ENDSTATIONNAME,ROUNDEDENDTIME,COUNTEVENTS,ENDDATE,ENDDAYNUM,ENDMONTHNUM,ENDTIME,ENDHOUR,ENDMINUTE,DAYOFWEEK
3802935,Métro de la Savane (Décarie / de Sorel),2023-10-27 23:30:00,2,2023-10-27,27,10,23:30:00,23,30,4
5549438,Ste-Famille / des Pins,2022-05-06 22:30:00,2,2022-05-06,6,5,22:30:00,22,30,4
6736072,des Seigneurs / Notre-Dame,2022-09-07 15:00:00,2,2022-09-07,7,9,15:00:00,15,0,2
6556621,de la Commune / Place Jacques-Cartier,2022-08-01 04:30:00,1,2022-08-01,1,8,04:30:00,4,30,0
6299011,de Maisonneuve / Mansfield (sud),2022-07-11 19:00:00,3,2022-07-11,11,7,19:00:00,19,0,0
...,...,...,...,...,...,...,...,...,...,...
89398,26e avenue / Beaubien,2022-08-16 16:00:00,1,2022-08-16,16,8,16:00:00,16,0,1
2383487,Lacombe / Victoria,2023-06-20 19:30:00,1,2023-06-20,20,6,19:30:00,19,30,1
3344321,Métro Honoré-Beaugrand (Sherbrooke / Honoré-Be...,2023-09-25 20:00:00,3,2023-09-25,25,9,20:00:00,20,0,0
4904275,Square Victoria (Viger / du Square-Victoria),2023-10-14 13:30:00,5,2023-10-14,14,10,13:30:00,13,30,5


In [ ]:
#export to excel
incoming_bikes.head(100000).to_excel('incoming_bikes.xlsx')
outgoing_bikes.head(100000).to_excel('outgoing_bikes.xlsx')
